In [2]:
with open('wnut17train.conll','r') as f:
    train=f.readlines()
with open('emerging.test.conll','r',encoding='utf8') as f:
    test=f.readlines()
#reading train and test files

In [3]:
from more_itertools import split_after
import nltk

In [6]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [7]:
def structure(sentences):
    sent=[]
    splits=list(split_after(sentences,lambda x:x=="\t\n" or x=='\n'))
    #splitting the sentence in \t\n or \n as they appear in the raw dataset
    for i in splits:
        s=[]
        for j in i[:-1]:
            word,label=j.strip('\n').split('\t')#removing the \n and splitting on \t
            tag=nltk.pos_tag([word])[0][1] #added pos_tag
            tup=(word,tag,label)#made it in a tuple of 3
            s.append(tup)
        sent.append(s)
return sent 

In [8]:
train=structure(train)
test=structure(test)
print(len(train),len(test))

3394 1287


In [9]:
# functions of sentence representations for sequence labelling
def word2features(sent, i):
    
    word = sent[i][0]
    postag = sent[i][1]


    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        # Indicate that it is the 'beginning of a document'
        features['BOS'] = True
        
    
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        # Features for words that are not at the end of a document
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [10]:
sample_sentence = " ".join([s for s,c,d in train[0]])
print(sample_sentence)
X_train = [sent2features(s) for s in train]
y_train = [sent2labels(s) for s in train]

X_test = [sent2features(s) for s in test]
y_test = [sent2labels(s) for s in test]

@paulwalk It 's the view from where I 'm living for two weeks . Empire State Building = ESB . Pretty bad storm here last evening .


In [11]:
!pip install sklearn_crfsuite
import sklearn_crfsuite
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)


In [12]:
crf.fit(X_train, y_train)

E:\ProgramData\Anaconda3\envs\deeplearn_course\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [13]:
labels = list(crf.classes_)
from sklearn_crfsuite import metrics

y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)

0.9291458549484162

In [24]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

E:\ProgramData\Anaconda3\envs\deeplearn_course\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=['O', 'B-corporation', 'I-corporation', 'B-creative-work', 'I-creative-work', 'B-group', 'I-group', 'B-location', 'I-location', 'B-person', 'I-person', 'B-product', 'I-product'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


                 precision    recall  f1-score   support

              O      0.948     0.995     0.971     21934
  B-corporation      0.000     0.000     0.000        62
  I-corporation      0.000     0.000     0.000        16
B-creative-work      0.091     0.008     0.014       131
I-creative-work      0.136     0.017     0.031       172
        B-group      0.200     0.052     0.082        97
        I-group      0.350     0.179     0.237        39
     B-location      0.299     0.250     0.272       104
     I-location      0.360     0.300     0.327        30
       B-person      0.455     0.135     0.208       333
       I-person      0.431     0.237     0.306        93
      B-product      0.000     0.000     0.000        50
      I-product      0.125     0.056     0.077        36

      micro avg      0.938     0.950     0.944     23097
      macro avg      0.261     0.171     0.194     23097
   weighted avg      0.913     0.950     0.929     23097

